<a href="https://colab.research.google.com/github/bala1802/ERA-Session-15/blob/main/TransformersNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/Session-15-Working

/content/drive/MyDrive/Session-15-Working


In [ ]:
# !git clone https://github.com/bala1802/ERA-Session-15.git

Cloning into 'ERA-Session-15'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 113 (delta 66), reused 74 (delta 30), pack-reused 0
Receiving objects: 100% (113/113), 23.58 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [ ]:
cd ERA-Session-15/

/content/drive/MyDrive/Session-15-Working/ERA-Session-15


In [ ]:
!pip install datasets --q
!pip install transformers --q
!pip install torchmetrics --q
!pip install tensorboard --q
!pip install lightning==2.0.1 --q
!pip install seaborn --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger,CSVLogger
import torch
import torch.nn as nn
import torchmetrics
import warnings
warnings.filterwarnings("ignore")
import logging
from torch.optim.lr_scheduler import OneCycleLR
import seaborn as sn
import pandas as pd

In [ ]:
from model import build_transformer
from config import get_config
from train import get_ds
from transformers import DataCollatorWithPadding


warnings.filterwarnings("ignore")

logger = logging.getLogger("Transformer")
logger.setLevel(level=logging.INFO)
fileHandler = logging.FileHandler(filename='prediction.log')
fileHandler.setLevel(level=logging.INFO)
logger.addHandler(fileHandler)

cfg = get_config()
cfg['preload'] = None
cfg['num_epochs'] = 40
cfg['batch_size'] = 16

In [ ]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(cfg)

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
class LitTr(pl.LightningModule):
    def __init__(self,cfg,tokenizer_src,tokenizer_tgt):
        super().__init__()

        vocab_src_len = tokenizer_src.get_vocab_size()
        vocab_tgt_len = tokenizer_tgt.get_vocab_size()
        self.model    = build_transformer(vocab_src_len, vocab_tgt_len, cfg['seq_len'], cfg['seq_len'], d_model=cfg['d_model'])
        self.loss_fn  = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
        self.BATCH_SIZE = cfg['batch_size']
        self.num_epochs = cfg['num_epochs']
        self.lr = cfg['lr']
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.seq_len = cfg['seq_len']
        self.expected = []
        self.predicted = []
        self.initial_epoch = 0
        self.save_hyperparameters()
        self.train_loss = []


    def forward(self, x):

        return self.model(x)

    def training_step(self, batch, batch_idx):

        encoder_input = batch['encoder_input']
        decoder_input = batch['decoder_input']
        encoder_mask  = batch['encoder_mask']
        decoder_mask  = batch['decoder_mask']

        # Run the tensors through the encoder, decoder and the projection layer
        encoder_output = self.model.encode(encoder_input, encoder_mask) # (b, seq_len, d_model)
        decoder_output = self.model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
        proj_output = self.model.project(decoder_output) # ( b, seq_len, vocab_size)

        label = batch['label']

        loss = self.loss_fn(proj_output.view(-1, self.tokenizer_tgt.get_vocab_size()), label.view(-1))
        self.train_loss.append(loss)

        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True,logger=True)
        return loss

    def casual_mask(self, size):
        mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
        return(mask == 0)

    def greedy_decode(self,source, source_mask):

        sos_idx = self.tokenizer_tgt.token_to_id('[SOS]')
        eos_idx = self.tokenizer_tgt.token_to_id('[EOS]')

        encoder_output = self.model.encode(source, source_mask)
        decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(source)

        while True:
            if decoder_input.size(1) == self.seq_len:
              break

            decoder_mask = self.casual_mask(decoder_input.size(1)).type_as(source_mask)

            out = self.model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

            prob = self.model.project(out[:,-1])
            _,next_word = torch.max(prob, dim=1)

            decoder_input = torch.cat(
                [decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item())], dim=1
            )

            if next_word == eos_idx:
              break

        return decoder_input.squeeze(0)

    def on_train_epoch_end(self):

        self.log('loss', torch.stack(self.train_loss).mean(), on_epoch=True, logger=True)
        print(f"Epoch {self.current_epoch} : Loss Mean - {torch.stack(self.train_loss).mean()}")
        self.train_loss.clear()

    def evaluate(self, batch, stage=None):

        encoder_input = batch["encoder_input"]
        encoder_mask  = batch["encoder_mask"]


        model_out = self.greedy_decode(encoder_input, encoder_mask)

        source_text = batch["src_text"][0]
        target_text = batch["tgt_text"][0]


        model_out_text = self.tokenizer_tgt.decode(model_out.detach().cpu().numpy())

        logger.info(f"SOURCE - {source_text}")
        logger.info(f"TARGET - {target_text}")
        logger.info(f"PREDICTED - {model_out_text}")
        logger.info("=============================================================")

        self.expected.append(target_text)
        self.predicted.append(model_out_text)


    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def on_validation_epoch_end(self):
        metric = torchmetrics.CharErrorRate()
        cer = metric(self.predicted, self.expected)
        self.log('validation_cer', cer, prog_bar=True, on_epoch=True, logger=True)


        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        wer = metric(self.predicted, self.expected)
        self.log('validation_wer', wer, prog_bar=True, on_epoch=True, logger=True)

        # Compute the BLEU metric
        metric = torchmetrics.BLEUScore()
        bleu = metric(self.predicted, self.expected)
        self.log('validation_bleu', bleu, prog_bar=True, on_epoch=True, logger=True)

        self.expected.clear()
        self.predicted.clear()

    def optimizer_zero_grad(self, epoch, batch_idx, optimizer):
        optimizer.zero_grad(set_to_none=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = self.lr, eps = 1e-9)
        scheduler = OneCycleLR(
                        optimizer,
                        max_lr= 10**-3,
                        pct_start = 1/10,
                        epochs=self.trainer.max_epochs,
                        steps_per_epoch=len(train_dataloader),
                        div_factor=10,
                        three_phase=True,
                        final_div_factor=10,
                        anneal_strategy='linear'
                    )
        return {
             "optimizer": optimizer,
             "lr_scheduler": {"scheduler": scheduler, "interval": "step"},
             }

In [ ]:
trainer = pl.Trainer(log_every_n_steps=1,
                     #accumulate_grad_batches=5,
                     limit_val_batches=2,
                     check_val_every_n_epoch=1,
                     max_epochs = cfg['num_epochs'],
                     accelerator='auto',
                     devices='auto',
                     strategy='auto',
                     #precision="bf16", #"16-mixed",
                     logger=[TensorBoardLogger("logs/", name="transformer-bilin"),CSVLogger(save_dir="logs/")],
    )

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
model = LitTr(cfg,tokenizer_src,tokenizer_tgt)
trainer.fit(model, train_dataloader ,val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type             | Params
---------------------------------------------
0 | model   | Transformer      | 56.3 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
56.3 M    Trainable params
0         Non-trainable params
56.3 M    Total params
225.350   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type             | Params
---------------------------------------------
0 | model   | Transformer      | 56.3 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
56.3 M    Trainable params
0         Non-trainable params
56.3 M    Total params
225.350   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

INFO:Transformer:SOURCE - 'Never, never!' said Julien, losing courage; 'I would let myself be brayed in a mortar sooner than compose three sentences; I am a corpse, my dear fellow, expect nothing more of me.
INFO:Transformer:TARGET - – Jamais, jamais ! dit Julien découragé ; je me ferais plutôt piler dans un mortier que de composer trois phrases ; je suis un cadavre, mon cher, n’espérez plus rien de moi.
INFO:Transformer:PREDICTED - contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent contractèrent con

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:Transformer:SOURCE - "I'm damned if I don't fling myself down the shaft before you!"
INFO:Transformer:TARGET - —Je me fous dans le puits devant vous!
INFO:Transformer:PREDICTED - -- Je suis si je ne vous pas avant que vous ne vous pas !
INFO:Transformer:=============================================================
INFO:Transformer:SOURCE - "'Oh,' returned the fairy, 'that does not signify! Here is a talisman will remove all difficulties;' and she held out a pretty gold ring.
INFO:Transformer:TARGET - «Oh! répondit la fée, peu importe; voilà un talisman qui lèvera toutes les difficultés.» Et elle me montra un bel anneau d'or.
INFO:Transformer:PREDICTED - -- Oh ! reprit le pays , c ' est qu ' il ne soit pas un .
INFO:Transformer:=============================================================


Epoch 0 : Loss Mean - 5.294425010681152


In [ ]:
metrics = pd.read_csv("logs/lightning_logs/version_0/metrics.csv")
del metrics["step"]
metrics.set_index("epoch", inplace=True)
display(metrics.dropna(axis=1, how="all").head())
sn.relplot(data=metrics, kind="line")